In [1]:
import pandas as pd
import numpy as np
from snowflake.sqlalchemy import URL
import datetime as datetime
import warnings
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

/Users/keyurpethad/anaconda3/lib/python3.11/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
month = "Dec'23"
prev_month = "Nov'23"
two_months_ago = "Oct'23"
three_months_ago = "Sep'23"
month_start = datetime.datetime(2023,12,1)

In [3]:
master_calling = pd.read_excel("PL Loan Book Jan'24.xlsx", header=2)

In [4]:
master_calling.head()

,niro_user_id,Niro Opportunity ID,Disbursal Partner Name,Platform Partner Name,Lender Loan Id,Disbursement Month,Month Number,Disbursement Year,Phone Numbers,Disbursement Date,Presentation Lag,Gross PF Charged,PF Refund,Final Proc Fees,GST,Pre-EMI,Disbursed Amount,Loan Amount (all inclusive),Loan Amount Rank,Interest Rate,Tenor In Months,Tenor Rank,PF %,Cost of Funds,Cost of Funds (New),Spread,Monthly Emi,Monthly Income (Net Take Home),Income Rank,Income source,Self Employed,Nature of Business,# Years in Employment,Residence Ownership Type,Owned,Purpose of Loan,CIBIL Score,CIBIL Band,CIBIL Score Rank,Risk Band,Risk Segmentation,Disbursement TAT,City,State,Pin Code,Tier,Gender,Male,Female,Age,customer_router,CPV Initiated,Lead Name,Loan Status,Foreclosure Date,Foreclosure Details,NPA Details,NPA setteled Status,Write-off Month,Feb Presentation,Mar Presentation,Apr Presentation,May Presentation,Jun Presentation,Jul Presentation,Aug Presentation,Sep Presentation,Oct Presentation,Nov Presentation,Dec Presentation,Jan'23 Presentation,Feb'23 Presentation,Mar'23 Presentation,Apr'23 Presentation,May'23 Presentation,Jun'23 Presentation,July'23 Presentation,Aug'23 Presentation,Sep'23 Presentation,Oct'23 Presentation,Nov'23 Presentation,Dec'23 Presentation,Jan'24 Presentation,Feb DPD,Mar DPD,Apr DPD,May DPD,Jun DPD,Jul DPD,Aug DPD,Sep DPD,Oct DPD,Nov DPD,Dec DPD,Jan'23 DPD,Feb'23 DPD,Mar'23 DPD,Apr'23 DPD,May'23 DPD,Jun'23 DPD,July'23 DPD,Aug'23 DPD,Sep'23 DPD,Oct'23 DPD,Nov'23 DPD,Dec'23 DPD,Mar DPD Rank,PayU'July'23 DPD,PayU'Aug'23 DPD,PayU'Sep'23 DPD,PayU'Oct'23 DPD,PayU'Nov'23 DPD,PayU'Dec'23 DPD,Sep DPD Rank,Jan - POS,Feb - POS,Mar - POS,Apr - POS,May - POS,Jun - POS,Jul - POS,Aug - POS,Sep - POS,Oct - POS,Nov - POS,Dec - POS,Jan'23 - POS,Feb'23 - POS,Mar'23 - POS,Apr'23 - POS,May'23 - POS,Jun'23 - POS,July'23 - POS,Aug'23 - POS,Sep'23 - POS,Oct'23 - POS,Nov'23 - POS,Dec'23 - POS,Int * Loan Amount (all Incl),CoF * Loan Amount (all Incl),Tenor * Loan Amount (all Incl),PF * Loan Amount (all Incl),Spread * Loan Amount (all Incl),FOIR,Ok/Problem Areas,Repeat Cases,DPD 90+ Months,DPD 60+ Months,90+ Amount,Feb - Repaid,Mar - Repaid,Apr - Repaid,May - Repaid,Jun - Repaid,Jul - Repaid,Aug - Repaid,Sep - Repaid,Oct - Repaid,Nov - Repaid,Dec - Repaid,Jan'23 - Repaid,Feb'23 - Repaid,Mar'23 - Repaid,Apr'23 - Repaid,May'23 - Repaid,Jun'23 - Repaid,July'23 - Repaid,Aug'23 - Repaid,Sep'23 - Repaid,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0f100756-11ae-4472-bd29-4fdbd6c37fdb,liquiloans,quikr,499217,2021-12-31,1,2021,9900368067,2021-12-31,Yes,2366,0,2366,426,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069,26000,3,Salaried,0,'-',1,Rented,0,Others,764,760+,8,CAT-D,True,< 24 hrs,Bangalore,KARNATAKA,560085,Tier 1,Female,0,1,39,PL_OLD_PREAPPROVED,Yes,VINUTHA HONNE GOWDA,Closed,NaT,NaN,NaN,No,-,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,NaN,-,-,-,-,-,-,NaN,52797.0,48915.93250,44947.540981,40889.860653,36740.882518,32498.552375,28160.769803,23722.160542,19190.208334,14553.208334,9811.208334,4962.960521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1425519.0,918667.8,633564,2366.0,506851.2,0.44,OK Area,1.0,-,-,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31,1.0
1,e85b616e-e54b-4d67-b929-5f4b612404e2,188ec580-5b20-435b-8b02-b1fd0c439c6b,liquiloans,quikr,500342,2022-01-31,2,2022,8765513171,2022-01-04,No,2366,0,2366,426,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069,20000,2,Salaried,0,'-',1.5,Owned,1,Marriage,746,740-760,7,CAT-D,True,< 24 hrs,Lucknow,UTTAR PRADESH,226029,Tier 2,Male,1,0,37,PL_OLD_PREAPPROVED,Yes,AKHILESH KUMAR YADAV,Foreclosed,2022-09-12,NaN,NaN,No,-,Cleared,Cleared,Cleared,Cleared,BOUNCED,BOUNCED,Bounce

In [5]:
master_calling['disb_dt'] = pd.to_datetime(master_calling['Disbursement Date'], format= '%d-%b-%y', errors = 'coerce')
master_calling['disb_yymm'] = master_calling['disb_dt'].dt.to_period('M').dt.to_timestamp()

master_calling['months_since_disb'] = (12*(month_start.year - master_calling['disb_yymm'].dt.year)+\
                           (month_start.month - master_calling['disb_yymm'].dt.month)+1).astype(int)
master_calling['PF'] = pd.to_numeric(master_calling["PF %"].astype(str).str.replace(" ","").str.replace("%","").str.strip())

#### EMI date Clearance type

In [6]:
master_calling[month + " clearance status"] = np.where(master_calling[month + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[month + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[month + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[month + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[month + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[month + " Presentation"])))))

master_calling[prev_month + " clearance status"] = np.where(master_calling[prev_month + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[prev_month + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[prev_month + " Presentation"])))))
                                                
master_calling[two_months_ago + " clearance status"] = np.where(master_calling[two_months_ago + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[two_months_ago + " Presentation"])))))

master_calling[three_months_ago + " clearance status"] = np.where(master_calling[three_months_ago + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[three_months_ago + " Presentation"])))))

#### Pre presentation bucket

In [7]:
master_calling[prev_month + " DPD"].unique()

array(['-', 'NPA', 'NPA (Unsettled)', 1, '30+', '60+', 12, 20, 9, '1-29+',
       'Settled', '90+', 3, 4, 6, 5, 8, 7, 2, 26, 28, 13, 'Closed', 11,
       16, 18, 21, 15, 27, 22, 19, 25, 10, 14, 23,
       'Settled (partial paid)', 24, 17, 0, -7, 'Foreclosed'],
      dtype=object)

In [9]:
master_calling[month + " Presentation bucket"] = np.where(master_calling[prev_month + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[prev_month + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[prev_month + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[prev_month + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[prev_month + " DPD"].isin(['90+']), 'E) Bkt 3+',
                                                np.where(master_calling[prev_month + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'E) Bkt 3+',
                                                np.where(master_calling[prev_month + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'F) Closed',master_calling[prev_month + " DPD"])))))))

master_calling[prev_month + " Presentation bucket"] = np.where(master_calling[two_months_ago + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[two_months_ago + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[two_months_ago + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['90+']), 'E) Bkt 3+',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'E) Bkt 3+',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'F) Closed',master_calling[two_months_ago + " DPD"])))))))

master_calling[two_months_ago + " Presentation bucket"] = np.where(master_calling[three_months_ago + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[three_months_ago + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[three_months_ago + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['90+']), 'E) Bkt 3+',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'E) Bkt 3+',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'F) Closed',master_calling[three_months_ago + " DPD"])))))))


### TAT calculation

In [10]:
month_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=month)
prev_month_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=prev_month)
two_months_ago_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=two_months_ago)
three_months_ago_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=three_months_ago)

In [11]:
month_df['EMI Submission date'] = pd.to_datetime(month_df['EMI Submission date'])
month_df['Collection Date'] = pd.to_datetime(month_df['Collection Date'], errors='coerce')

prev_month_df['EMI Submission date'] = pd.to_datetime(prev_month_df['EMI Submission date'])
prev_month_df['Collection Date'] = pd.to_datetime(prev_month_df['Collection Date'], errors='coerce')

two_months_ago_df['EMI Submission date'] = pd.to_datetime(two_months_ago_df['EMI Submission date'])
two_months_ago_df['Collection Date'] = pd.to_datetime(two_months_ago_df['Collection Date'], errors='coerce')

three_months_ago_df['EMI Submission date'] = pd.to_datetime(three_months_ago_df['EMI Submission date'])
three_months_ago_df['Collection Date'] = pd.to_datetime(three_months_ago_df['Collection Date'], errors='coerce')

In [12]:
month_df[month + ' TAT'] = np.where(month_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(month_df['Bounce Paid/unpaid'].isin(['Settled', 'Settled (partial paid)']), 9999,
                        np.where(month_df['Bounce Paid/unpaid'].isin(['Advance']), -1,
                        np.where(month_df['Bounce Paid/unpaid'].isin(['Paid']),(month_df['Collection Date'] - month_df['EMI Submission date']).dt.days,
                        np.nan))))

prev_month_df[prev_month + ' TAT'] = np.where(prev_month_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(prev_month_df['Bounce Paid/unpaid'].isin(['Settled', 'Settled (partial paid)']), 9999,
                        np.where(prev_month_df['Bounce Paid/unpaid'].isin(['Advance']), -1,
                        np.where(prev_month_df['Bounce Paid/unpaid'].isin(['Paid']),(prev_month_df['Collection Date'] - prev_month_df['EMI Submission date']).dt.days,
                        np.nan)))) 

two_months_ago_df[two_months_ago + ' TAT'] = np.where(two_months_ago_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(two_months_ago_df['Bounce Paid/unpaid'].isin(['Settled', 'Settled (partial paid)']), 9999,
                        np.where(two_months_ago_df['Bounce Paid/unpaid'].isin(['Advance']), -1,
                        np.where(two_months_ago_df['Bounce Paid/unpaid'].isin(['Paid']),(two_months_ago_df['Collection Date'] - two_months_ago_df['EMI Submission date']).dt.days,
                        np.nan))))

three_months_ago_df[three_months_ago + ' TAT'] = np.where(three_months_ago_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(three_months_ago_df['Bounce Paid/unpaid'].isin(['Settled', 'Settled (partial paid)']), 9999,
                        np.where(three_months_ago_df['Bounce Paid/unpaid'].isin(['Advance']), -1,
                        np.where(three_months_ago_df['Bounce Paid/unpaid'].isin(['Paid']),(three_months_ago_df['Collection Date'] - three_months_ago_df['EMI Submission date']).dt.days,
                        np.nan))))

month_df[month + ' TAT_bkt'] = np.where(month_df[month + ' TAT'] < -60, np.nan,
                                            np.where(month_df[month + ' TAT'] < 0, 'A) <0',
                                            np.where(month_df[month + ' TAT'] == 0, 'B) 0',
                                            np.where(month_df[month + ' TAT'] <= 10, 'C) 1-10',
                                            np.where(month_df[month + ' TAT'] <= 20, 'D) 10 - 20',
                                            np.where(month_df[month + ' TAT'] <= 29, 'E) 21 - 29',
                                            np.where(month_df[month + ' TAT'] == 999, 'F) Unpaid',
                                            np.where(month_df[month + ' TAT'] == 9999, 'G) Settled', month_df[month + ' TAT']))))))))

prev_month_df[prev_month + ' TAT_bkt'] = np.where(prev_month_df[prev_month + ' TAT'] < -60, np.nan,
                                            np.where(prev_month_df[prev_month + ' TAT'] < 0, 'A) < 0',
                                            np.where(prev_month_df[prev_month + ' TAT'] == 0, 'B) 0',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 10, 'C) 1-10',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 20, 'D) 10 - 20',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 29, 'E) 21 - 29',
                                            np.where(prev_month_df[prev_month + ' TAT'] == 999, 'F) Unpaid',
                                            np.where(prev_month_df[prev_month + ' TAT'] == 9999, 'G) Settled', prev_month_df[prev_month + ' TAT']))))))))

two_months_ago_df[two_months_ago + ' TAT_bkt'] = np.where(two_months_ago_df[two_months_ago + ' TAT'] < -60, np.nan,
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] < 0, 'A) <0',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] == 0, 'B) 0',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 10, 'C) 1-10',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 20, 'D) 10 - 20',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 29, 'E) 21 - 29',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] == 999, 'F) Unpaid',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] == 9999, 'G) Settled', two_months_ago_df[two_months_ago + ' TAT']))))))))

three_months_ago_df[three_months_ago + ' TAT_bkt'] = np.where(three_months_ago_df[three_months_ago + ' TAT'] < -60, np.nan,
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] < 0, 'A) <0',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] == 0, 'B) 0',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 10, 'C) 1-10',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 20, 'D) 10 - 20',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 29, 'E) 21 - 29',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] == 999, 'F) Unpaid',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] == 9999, 'G) Settled', three_months_ago_df[three_months_ago + ' TAT']))))))))

In [13]:
master_calling.head(2)

,niro_user_id,Niro Opportunity ID,Disbursal Partner Name,Platform Partner Name,Lender Loan Id,Disbursement Month,Month Number,Disbursement Year,Phone Numbers,Disbursement Date,Presentation Lag,Gross PF Charged,PF Refund,Final Proc Fees,GST,Pre-EMI,Disbursed Amount,Loan Amount (all inclusive),Loan Amount Rank,Interest Rate,Tenor In Months,Tenor Rank,PF %,Cost of Funds,Cost of Funds (New),Spread,Monthly Emi,Monthly Income (Net Take Home),Income Rank,Income source,Self Employed,Nature of Business,# Years in Employment,Residence Ownership Type,Owned,Purpose of Loan,CIBIL Score,CIBIL Band,CIBIL Score Rank,Risk Band,Risk Segmentation,Disbursement TAT,City,State,Pin Code,Tier,Gender,Male,Female,Age,customer_router,CPV Initiated,Lead Name,Loan Status,Foreclosure Date,Foreclosure Details,NPA Details,NPA setteled Status,Write-off Month,Feb Presentation,Mar Presentation,Apr Presentation,May Presentation,Jun Presentation,Jul Presentation,Aug Presentation,Sep Presentation,Oct Presentation,Nov Presentation,Dec Presentation,Jan'23 Presentation,Feb'23 Presentation,Mar'23 Presentation,Apr'23 Presentation,May'23 Presentation,Jun'23 Presentation,July'23 Presentation,Aug'23 Presentation,Sep'23 Presentation,Oct'23 Presentation,Nov'23 Presentation,Dec'23 Presentation,Jan'24 Presentation,Feb DPD,Mar DPD,Apr DPD,May DPD,Jun DPD,Jul DPD,Aug DPD,Sep DPD,Oct DPD,Nov DPD,Dec DPD,Jan'23 DPD,Feb'23 DPD,Mar'23 DPD,Apr'23 DPD,May'23 DPD,Jun'23 DPD,July'23 DPD,Aug'23 DPD,Sep'23 DPD,Oct'23 DPD,Nov'23 DPD,Dec'23 DPD,Mar DPD Rank,PayU'July'23 DPD,PayU'Aug'23 DPD,PayU'Sep'23 DPD,PayU'Oct'23 DPD,PayU'Nov'23 DPD,PayU'Dec'23 DPD,Sep DPD Rank,Jan - POS,Feb - POS,Mar - POS,Apr - POS,May - POS,Jun - POS,Jul - POS,Aug - POS,Sep - POS,Oct - POS,Nov - POS,Dec - POS,Jan'23 - POS,Feb'23 - POS,Mar'23 - POS,Apr'23 - POS,May'23 - POS,Jun'23 - POS,July'23 - POS,Aug'23 - POS,Sep'23 - POS,Oct'23 - POS,Nov'23 - POS,Dec'23 - POS,Int * Loan Amount (all Incl),CoF * Loan Amount (all Incl),Tenor * Loan Amount (all Incl),PF * Loan Amount (all Incl),Spread * Loan Amount (all Incl),FOIR,Ok/Problem Areas,Repeat Cases,DPD 90+ Months,DPD 60+ Months,90+ Amount,Feb - Repaid,Mar - Repaid,Apr - Repaid,May - Repaid,Jun - Repaid,Jul - Repaid,Aug - Repaid,Sep - Repaid,Oct - Repaid,Nov - Repaid,Dec - Repaid,Jan'23 - Repaid,Feb'23 - Repaid,Mar'23 - Repaid,Apr'23 - Repaid,May'23 - Repaid,Jun'23 - Repaid,July'23 - Repaid,Aug'23 - Repaid,Sep'23 - Repaid,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,disb_dt,disb_yymm,months_since_disb,PF,Dec'23 clearance status,Nov'23 clearance status,Oct'23 clearance status,Sep'23 clearance status,Dec'23 Presentation bucket,Nov'23 Presentation bucket,Oct'23 Presentation bucket
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0f100756-11ae-4472-bd29-4fdbd6c37fdb,liquiloans,quikr,499217,2021-12-31,1,2021,9900368067,2021-12-31,Yes,2366,0,2366,426,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069,26000,3,Salaried,0,'-',1,Rented,0,Others,764,760+,8,CAT-D,True,< 24 hrs,Bangalore,KARNATAKA,560085,Tier 1,Female,0,1,39,PL_OLD_PREAPPROVED,Yes,VINUTHA HONNE GOWDA,Closed,NaT,NaN,NaN,No,-,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,NaN,-,-,-,-,-,-,NaN,52797.0,48915.9325,44947.540981,40889.860653,36740.882518,32498.552375,28160.769803,23722.160542,19190.208334,14553.208334,9811.208334,4962.960521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1425519.0,918667.8,633564,2366.0,506851.2,0.44,OK Area,1.0,-,-,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31,1.0,2021-12-31,2021-12-01,25,0.044813,E) Closed,E) Closed,E) Closed,E) Closed,A) Bkt 0,A) Bkt 0,A) Bkt 0
1,e85b616e-e54b-4d67-b929-5f4b612404e2,188ec580-5b20-435b-8b02-b1fd0c439c6b,liquiloans,quikr,500342,2022-01-31,2,2022,8765513171,2022-01-04,No,2366,0,2366,426

In [14]:
prev_month_df.head(1)

,Opportunity ID,Loan Id,EMI Submission date,Bounced cases,Bounce Paid/unpaid,Collection Date,DPD,Nov'23 TAT,Nov'23 TAT_bkt
0,0f100756-11ae-4472-bd29-4fdbd6c37fdb,499217,2023-11-02,Closed,Closed,NaT,0,NaN,nan


In [15]:
master_calling = master_calling.merge(month_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", month + ' TAT', month + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") \
    .merge(prev_month_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", prev_month + ' TAT', prev_month + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") \
    .merge(two_months_ago_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", two_months_ago + ' TAT', two_months_ago + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") \
    .merge(three_months_ago_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", three_months_ago + ' TAT', three_months_ago + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") 

#### Creating Collection Data

In [16]:
master_calling.shape

(40070, 193)

In [17]:
cols = ['Lender Loan Id', 'Presentation Lag', 'Tenor In Months', 'Tenor Rank', 'Disbursed Amount', 'months_since_disb', 'PF', 'Monthly Income (Net Take Home)', 'Monthly Emi', 'Income Rank', 'City', 'State', 'Tier', 'Loan Status', 'FOIR', 'Ok/Problem Areas', 'Repeat Cases', 'Phone Numbers'] 
master_calling1 = master_calling[['niro_user_id', 'Niro Opportunity ID', month + " clearance status", month + " Presentation bucket", prev_month + ' TAT', prev_month + ' TAT_bkt', month + ' TAT_bkt', month + ' - POS'] + cols].rename(columns = {month + " clearance status" : "month_clearance_status", month + " Presentation bucket" : "month_presentation_bucket", prev_month + ' TAT': 'prev_month_TAT', prev_month + ' TAT_bkt': 'prev_month_TAT_bkt', month + ' TAT_bkt': 'month_TAT_bkt', month + ' - POS': 'month_POS'})

In [18]:
master_calling1[master_calling1["month_clearance_status"].isin(['B) Bounce', 'C) T-Bounce']) & master_calling1.month_TAT_bkt.isin(['nan', 'B) -10 to -1', 'A) <= -10'])].month_TAT_bkt.value_counts()

month_TAT_bkt
nan    2
Name: count, dtype: int64

In [19]:
master_calling1[master_calling1["month_clearance_status"].isin(['B) Bounce', 'C) T-Bounce'])].shape

(8702, 26)

In [20]:
master_calling1 = master_calling1[master_calling1["month_clearance_status"].isin(['B) Bounce', 'C) T-Bounce']) & ~master_calling1.month_TAT_bkt.isin(['nan', 'B) -10 to -1', 'A) <= -10'])]

In [21]:
master_calling1.loc[master_calling1.month_TAT_bkt.isin(["F) Unpaid", "G) Settled"]), 'Event'] = 1
master_calling1.loc[master_calling1.month_TAT_bkt.isin(['D) 10 - 20', 'E) 21 - 29', 'C) 1-10', 'B) 0', 'A) <0']), 'Event'] = 0

In [22]:
master_calling1.drop(columns = ['month_TAT_bkt'], inplace=True)

In [23]:
master_calling1.head()

,niro_user_id,Niro Opportunity ID,month_clearance_status,month_presentation_bucket,prev_month_TAT,prev_month_TAT_bkt,month_POS,Lender Loan Id,Presentation Lag,Tenor In Months,Tenor Rank,Disbursed Amount,months_since_disb,PF,Monthly Income (Net Take Home),Monthly Emi,Income Rank,City,State,Tier,Loan Status,FOIR,Ok/Problem Areas,Repeat Cases,Phone Numbers,Event
6,e9ed949b-e93b-42a2-ac21-2da7effa5564,8cf95828-6981-4f75-a766-f79769c8bd8a,B) Bounce,E) Bkt 3+,999.0,F) Unpaid,90887.788379,518818,No,18,3,180001,24,0.034993,30000,12295,3,Chennai,TAMIL NADU,Tier 1,NPA,0.42,Problem Area,1.0,9952744141,1.0
30,7b232688-b858-4018-9a29-d2ab52a39b76,b7d14eef-e5b4-4b55-9ee7-1c033f6ccc71,B) Bounce,E) Bkt 3+,999.0,F) Unpaid,11769.521788,583075,No,18,3,60003,23,0.039954,24000,4048,2,Thane,MAHARASHTRA,Tier 1,NPA,0.50,OK Area,1.0,9821845214,1.0
32,dceb0534-243c-4f67-94c3-ef8a0b45e6bc,cdf9f121-0176-4ca9-aa9d-f5d4048af74b,B) Bounce,A) Bkt 0,1.0,C) 1-10,12461.921883,589966,No,24,4,125003,23,0.034981,80000,6353,5,Hyderabad,ANDHRA PRADESH,Tier 1,Active,0.47,OK Area,1.0,9989185412,0.0
33,d49495cf-7cee-43fe-ba28-c6d4cee801a1,e345da0b-34fd-480c-bde5-4d94edd520bb,B) Bounce,C) Bkt 2,1.0,C) 1-10,48654.610670,587605,No,24,4,210001,23,0.014996,40000,10014,3,Kozhikode,KERALA,Tier 3,Active,0.44,OK Area,1.0,7021643837,1.0
34,83efdb11-99cc-4e7a-a1cd-6f0cbf4656f1,dd5f7140-b43c-4712-9ac6-ea4abe82a330,B) Bounce,E) Bkt 3+,999.0,F) Unpaid,26670.448080,589873,No,15,3,50004,23,0.039941,40000,4161,3,Bangalore,KARNATAKA,Tier 1,NPA,0.47,OK Area,1.0,7085197694,1.0


#### Advance Collection file

In [24]:
adv_coll_tag = pd.read_excel('advance collection detailing report.xlsx', sheet_name=month).rename(columns = {'opportunity_id':'Niro Opportunity ID', 'Code':'Code_'+month, 'Status':'Status_'+month, 'Code_meaning':'Code_meaning_'+month})
adv_coll_tag.head()

,Niro Opportunity ID,Code_Dec'23,EMI,Status_Dec'23,Code_meaning_Dec'23
0,cdf9f121-0176-4ca9-aa9d-f5d4048af74b,CTB,6353,Notcollected,Current month bounced and non dpd ( voucher )
1,f9fcd6d5-dcfc-4c64-8a00-c999a4fea08f,NTB-6N,8090,Notcollected,never bounced Minimum 6 months
2,e345da0b-34fd-480c-bde5-4d94edd520bb,CMU-DPD,10014,Notcollected,Current month bounce and DPD
3,2bef54d9-cca9-4f44-915a-bf039fc486ba,NTB-6N,4146,Notcollected,never bounced Minimum 6 months
4,ce06a9a0-8b47-44f8-af4a-fd6b0fda7224,CMU-DPD,3842,Notcollected,Current month bounce and DPD


In [25]:
adv_coll_tag.rename(columns = {'Code_'+month:'advance_collection_code_month', 'Status_'+month: 'advance_collection_status_month'}, inplace=True)

In [26]:
master_data = master_calling1.merge(adv_coll_tag[['Niro Opportunity ID', 'advance_collection_code_month']], on = 'Niro Opportunity ID', how='left')

In [27]:
master_data.head()

,niro_user_id,Niro Opportunity ID,month_clearance_status,month_presentation_bucket,prev_month_TAT,prev_month_TAT_bkt,month_POS,Lender Loan Id,Presentation Lag,Tenor In Months,Tenor Rank,Disbursed Amount,months_since_disb,PF,Monthly Income (Net Take Home),Monthly Emi,Income Rank,City,State,Tier,Loan Status,FOIR,Ok/Problem Areas,Repeat Cases,Phone Numbers,Event,advance_collection_code_month
0,e9ed949b-e93b-42a2-ac21-2da7effa5564,8cf95828-6981-4f75-a766-f79769c8bd8a,B) Bounce,E) Bkt 3+,999.0,F) Unpaid,90887.788379,518818,No,18,3,180001,24,0.034993,30000,12295,3,Chennai,TAMIL NADU,Tier 1,NPA,0.42,Problem Area,1.0,9952744141,1.0,NaN
1,7b232688-b858-4018-9a29-d2ab52a39b76,b7d14eef-e5b4-4b55-9ee7-1c033f6ccc71,B) Bounce,E) Bkt 3+,999.0,F) Unpaid,11769.521788,583075,No,18,3,60003,23,0.039954,24000,4048,2,Thane,MAHARASHTRA,Tier 1,NPA,0.50,OK Area,1.0,9821845214,1.0,NaN
2,dceb0534-243c-4f67-94c3-ef8a0b45e6bc,cdf9f121-0176-4ca9-aa9d-f5d4048af74b,B) Bounce,A) Bkt 0,1.0,C) 1-10,12461.921883,589966,No,24,4,125003,23,0.034981,80000,6353,5,Hyderabad,ANDHRA PRADESH,Tier 1,Active,0.47,OK Area,1.0,9989185412,0.0,CTB
3,d49495cf-7cee-43fe-ba28-c6d4cee801a1,e345da0b-34fd-480c-bde5-4d94edd520bb,B) Bounce,C) Bkt 2,1.0,C) 1-10,48654.610670,587605,No,24,4,210001,23,0.014996,40000,10014,3,Kozhikode,KERALA,Tier 3,Active,0.44,OK Area,1.0,7021643837,1.0,CMU-DPD
4,83efdb11-99cc-4e7a-a1cd-6f0cbf4656f1,dd5f7140-b43c-4712-9ac6-ea4abe82a330,B) Bounce,E) Bkt 3+,999.0,F) Unpaid,26670.448080,589873,No,15,3,50004,23,0.039941,40000,4161,3,Bangalore,KARNATAKA,Tier 1,NPA,0.47,OK Area,1.0,7085197694,1.0,NaN


#### Pre disbursal data

In [28]:
from sqlalchemy import create_engine


class extractorFromQuery:
    def __init__(self):
        self.credentials = {
            "account": "qfi-niro",
            "user": "keyur",
            "password": "Keyur@17061996",
            "warehouse": "NIRO_ANALYTICS_WAREHOUSE",
        }

    def queryExecutor(self, query: str, *args):
        engine = create_engine(URL(**self.credentials))
        res = pd.read_sql_query(query, engine, params=args)
        return res

In [29]:
disbursed_cases = """
    select 
    la.id,
    ou.category,
    la.principal_amount,
    la.interest_rate,
    o.attributed_partner as "supply",
    o.propensity,
    o.offer_type,
    o.max_amount,
    tu.income_source,
    tu.purpose_of_loan,
    tu.residence_ownership_type,
    tu.marital_status,
    cpv.cpv_type,
    l.name as "demand",
    ba.bank_name,
    CASE
        when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
        else date(la.loan_disbursed_date_by_ops)
    end as disbursement_date
    from niro_rds.niro_data.core__public__loan_application la
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__OFFERS o on o.id = la.offer_id
    left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = o.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__VERIFICATION_DETAILS vd on vd.id = la.verification_id 
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__TELE_PD_USER tu on vd.telepd_id::bigint = tu.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__CP_VERIFICATION cpv on cpv.id = vd.cpv_id::bigint
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LENDERS l on l.id = o.lender_uid
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__BANK_ACCOUNTS b on b.id = la.bank_account_id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__BANK_BRANCHES ba on b.ifsc_code = ba.ifsc_code  
    where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') and la.is_deleted = False and o.is_deleted = False;
"""
total_disbursed_cases = extractorFromQuery().queryExecutor(disbursed_cases)
total_disbursed_cases.shape

(45882, 16)

In [30]:
total_disbursed_cases['residence_ownership_type'] = total_disbursed_cases.residence_ownership_type.str.lower().replace({'family_owned':'family owned'}).replace({'owned':'self owned'}).replace({'pg':'rented'}).replace({'room sharing':'rented'}).replace({'ancestral property':'family owned'})

In [31]:
total_disbursed_cases['bank_name'].value_counts()

bank_name
HDFC BANK                                      9061
STATE BANK OF INDIA                            8943
ICICI BANK LIMITED                             4415
KOTAK MAHINDRA BANK LIMITED                    4032
AXIS BANK                                      3367
CANARA BANK                                    2281
UNION BANK OF INDIA                            2245
BANK OF BARODA                                 2202
PUNJAB NATIONAL BANK                           1574
INDIAN BANK                                    1261
BANK OF INDIA                                  1021
FEDERAL BANK                                    756
INDUSIND BANK                                   620
IDBI BANK                                       611
KARNATAKA BANK LIMITED                          530
BANK OF MAHARASHTRA                             432
IDFC FIRST BANK LTD                             422
YES BANK                                        285
CITY UNION BANK LIMITED                         219
IN

In [ ]:
disbursed_cases_with_cvattrs = """
            select * from (
            select 
            distinct la.id,
            pii.gender as bureau_gender,
            pii.pin as bureau_pincode,
            pii.calculated_age,
            pii.city as bureau_city,
            pii.state as bureau_state,
            pii.native_language,
            date_trunc("MONTH",date(od.created_at)) as createdate,
            od.current_salary,
            od.available_income,
            od.foir_dlq, od.foir_cibil, od.foir_trd, od.foir_mean, od.new_foir, 
            od.naps_score as naps_disb_score,
            rank()over(partition by od.user_id order by od.created_at asc) as rnk,
            CASE
                when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
                else date(la.loan_disbursed_date_by_ops)
            END as disbursement_date
            from NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LOAN_APPLICATION la
            left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = la.offer_id
            left join ANALYTICS.OFFERS.offer_details od on od.user_id = la.user_id and od.category = ou.category
            left join ANALYTICS.OFFERS.personal_info pii on od.pii_id = pii.pii_id
            where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') 
            and la.is_deleted = False
            and(la.is_deleted_at_source = false or la.is_deleted_at_source is null)
            and od.dec_reason = 'NOT DECLINED')
where rnk = 1;
"""
disbursed_cases = (
    extractorFromQuery()
    .queryExecutor(disbursed_cases_with_cvattrs)
    .drop(columns="disbursement_date")
)
disbursed_cases.shape

In [ ]:
disbursed_cases = disbursed_cases.drop_duplicates(subset=["id"])
predisbursal_data = total_disbursed_cases.merge(disbursed_cases, on="id", how="left").rename(
    columns={"id": "niro_opportunity_id"}
)
predisbursal_data.drop_duplicates(subset=["niro_opportunity_id"], inplace=True)
predisbursal_data.shape

In [ ]:
predisbursal_data['propensity'] = np.where(predisbursal_data['propensity'].isin(['HiiProp', 'High']), 'High',
                                  np.where(predisbursal_data['propensity'].isin(['MedProp', 'Med', 'Medium']), 'Medium',
                                  np.where(predisbursal_data['propensity'].isin(['LowProp', 'Low']), 'Low',None)))

In [ ]:
predisbursal_data.head()

In [ ]:
master_data.shape

In [ ]:
master_data = pd.merge(master_data.rename(columns = {'Niro Opportunity ID': 'niro_opportunity_id'}), predisbursal_data , on = 'niro_opportunity_id', how = 'inner')

In [ ]:
master_data.tail()

#### Scrub data

In [ ]:
scrub_data = pd.read_csv('Disbursed_cases_Scrub_05112023.csv')
scrub_data.head()

In [ ]:
scrub_data.shape

In [ ]:
column_name_map = {'AGG911': 'max_aggregate_bankcard_utilisation_l12m',
 'RVLR01': 'cc_util_revolving_l1m',
 'BCPMTSTR': 'bank_card_payment_category',
 'CV11': 'no_of_60p_accs_ever',
 'CV14': 'no_of_deduped_inquiries',
 'MT28S': 'total_cl_open_mortgage_trades_l12m',
 'MT33S': 'total_balance_open_mortgage_trades_l12m',
 'PL33S': 'total_bal_of_open_pl_l12m',
 'AT20S': 'months_since_oldest_trade',
 'MT01S': 'no_of_mortgage_trades',
 'BC02S': 'no_of_open_cc_trades',
 'BG01S': 'no_of_business_general_trades',
 'CV10': 'no_of_30p_accs_ever',
 'TRD': 'no_of_trades',
 'AT33A': 'total_bal_open_trades_l12m',
 'AU33S': 'total_bal_open_auto_trades_l12m',
 'CO04S180': 'months_since_recent_chargedoff180P',
 'AU28S': 'total_cl_open_auto_trades_l12m',
 'PL28S': 'total_cl_open_pl_trades_l12m',
 'CO01S180': 'no_of_chargedoff180P',
 'BC28S': 'total_cl_of_cc_trades_l12m',
 'CV12': 'no_of_90p_accs_ever',
 'CO05S': 'total_bal_of_chargedoff_trades',
 'G310S': 'worst_rating_l12m',
 'AGGS911': 'current_utilization_of_top_wallet_bankcard',
 'AT01S': 'no_of_trades_pl_2w_gl_cd',
 'AT33A_NE_CCOD': 'total_bal_open_trades_l12m_except_wo_cc_od',
 'CV14_12M': 'no_of_deduped_inquiries_l12m',
 'CV14_6M': 'no_of_deduped_inquiries_l6m',
 'CV14_3M': 'no_of_deduped_inquiries_l3m',
 'CV14_1M': 'no_of_deduped_inquiries_l1m',
 'G310S_24M': 'worst_rating_l24m',
 'G310S_6M': 'worst_rating_l6m',
 'G310S_3M': 'worst_rating_l3m',
 'G310S_1M': 'worst_rating_l1m',
 'CV11_24M': 'no_of_60p_accs_l24m',
 'CV11_12M': 'no_of_60p_accs_l12m',
 'G057S_1DPD_36M': 'no_of_trades_bounced_l36m',
 'G057S_1DPD_12M': 'no_of_trades_bounced_l12m',
 'BC106S_60DPD': 'no_of_cc_trades_60P_l24m',
 'BC107S_24M': 'no_of_30P_cc_l24m',
 'BC106S_60DPD_12M': 'no_of_cc_trades_60P_l12m',
 'BC107S_12M': 'no_of_30P_cc_l12m',
 'BC106S_LE_30DPD_12M': 'no_of_cc_lt30p_l12m',
 'BC09S_36M_HCSA_LE_30': 'no_cc_sanctionedamt_lt30K_l36m',
 'PL09S_36M_HCSA_LE_30': 'no_pl_sanctionedamt_lt30K_l36m',
 'AT09S_6M': 'no_tr_opened_l6m',
 'G310S_36M': 'worst_dpd_l36m',
 'AT33A_NE_WO': 'total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades',
 'AT09S_12M': 'no_tr_opened_l12m',
 'AT09S_3M': 'no_tr_opened_l3m',
 'CV13': 'per_of_30p_accs_ever',
 'CV24': 'total_payment_amount_bankcard_accs_l3m',
 'REVS904': 'max_revolving_monthly_spend_l12m',
 'CV20': 'total_monthly_obligations_l3m',
 'CV22': 'total_balance_bankcard_accs_l3m',
 'UL_TRD': 'no_of_trades_unsec',
 'CV21': 'total_payment_amount_l3m',
 'G310S_2M': 'worst_dpd_l2m'}

In [ ]:
scrub_data.rename(columns = column_name_map, inplace=True)

In [ ]:
scrub_data.head()

In [ ]:
def naps_score_bkt(row, var):
    if row[var] <= 689:
        return '<=689'
    elif row[var] <= 704:
        return '690-704'
    elif row[var] <= 713:
        return '705-713'
    elif row[var] <= 728:
        return '714-728'
    elif row[var] <= 742:
        return '723-742'
    elif row[var] <= 751:
        return '743-751'
    elif row[var] <= 770:
        return '752-770'
    elif row[var] <= 790:
        return '771-790'
    elif row[var] <= 818:
        return '791-818'
    else:
        return '819+'


#statename
#BCPMTSTR
scrub_data.rename(columns = {'CIBILTUSC3 Score Value':'CIBIL_score'}, inplace=True)

scrub_data['no_of_open_cc_trades_bkt'] = np.select(condlist=[scrub_data['no_of_open_cc_trades']<=0,
                                             scrub_data['no_of_open_cc_trades']<=2,
                                             scrub_data['no_of_open_cc_trades']>2,],choicelist=['No Credit Cards','1-2','3+'],default='No Credit Cards')

scrub_data['years_on_bureau_bkt'] = np.select(condlist=[scrub_data['months_since_oldest_trade']<=48,
                                             scrub_data['months_since_oldest_trade']>48,],choicelist=['<=4 Yrs','4+ Yrs'],default='None')

scrub_data['no_of_deduped_inquiries_bkt'] = np.select(condlist=[scrub_data['no_of_deduped_inquiries_l12m']<=1,
                                             scrub_data['no_of_deduped_inquiries_l12m']<=3,
                                             scrub_data['no_of_deduped_inquiries_l12m']<=6,
                                             scrub_data['no_of_deduped_inquiries_l12m']>6,],choicelist=['<=1','2-3',
                                                                                        '4-6','7+'],default='None')

scrub_data['Secured Loan_bkt'] = np.select(condlist=[scrub_data['Secured High Credit Sum']<=0,
                                             scrub_data['Secured High Credit Sum']<=200000,
                                             scrub_data['Secured High Credit Sum']<=750000,
                                             scrub_data['Secured High Credit Sum']>750000,],choicelist=['No Secured loan','Secured Loan < 2L',
                                                                                        'Secured Loan < 7.5L','Secured Loan 7.5L+'],
                                  default='No Secured loan')

# final['Income_bkt'] = np.select(condlist=[final['ctc']<=15000,
#                                             final['ctc']<=20000,
#                                             final['ctc']<=25000,
#                                              final['ctc']<=30000,
#                                              final['ctc']<=35000,
#                                             final['ctc']<=40000,
#                                             final['ctc']<=50000,
#                                             final['ctc']<=75000,
#                                              final['ctc']>75000],choicelist=['<=15000','15000-20000','20000-25000','25000-30000',
#                                                                                 '30000-35000','35000-40000','40000-50000',
#                                                                                 '50000-75000','75000+'],\
#                             default='None')

# final['FOIR_Band'] = np.select(condlist=[final['New_FOIR']<=35,
#                                              final['New_FOIR']<=50,
#                                              final['New_FOIR']<=60,
#                                             final['New_FOIR']>60],choicelist=['<=35%','35-50%',
#                                                                                         '50-60%','60-70%'],default='None')

scrub_data['cust_category'] = np.select(condlist=[((scrub_data['no_of_open_cc_trades']>=1) & (scrub_data['total_cl_of_cc_trades_l12m']>=50000))&((scrub_data['total_cl_open_auto_trades_l12m']>=200000)|(scrub_data['total_cl_open_mortgage_trades_l12m']>=1000000)),\
                                          ((scrub_data['no_of_open_cc_trades']>=1) & (scrub_data['total_cl_of_cc_trades_l12m']>=50000)),
                                          ((scrub_data['no_of_open_cc_trades']<=0) & (scrub_data['Unsecured High Credit Sum']>=50000))
                                            ],choicelist=['Carded 50K+ With Secured Loans','Carded 50K+',\
                                                          'Unsecured Loan 50K+'],
                                    default='None')

scrub_data['cust_hirisk'] = np.select(condlist=[((scrub_data['Unsecured High Credit Sum']<=82000) &
                                            (scrub_data['no_of_deduped_inquiries_l1m']>=2)&
                                           (scrub_data['no_of_trades_pl_2w_gl_cd']>=3))
                                            ],choicelist=[1],default=0)

# final['niro_user_id'] = final['user_id'].astype(str).str.strip()


In [ ]:
scrub_data.columns

In [ ]:
def preprocessor(df):

    try:
        df['CC_INACTIVE'] = np.select(condlist=[(df['bank_card_payment_category']=='INACTIVE')],choicelist=[1],default=0)
        df['CC_NOBC'] = np.select(condlist=[(df['bank_card_payment_category']=='NOBC')],choicelist=[1],default=0)
        df['CC_REVOLVER'] = np.select(condlist=[(df['bank_card_payment_category']=='REVOLVER')],choicelist=[1],default=0)
        df['CC_RVLRPLUS'] = np.select(condlist=[(df['bank_card_payment_category']=='RVLRPLUS')],choicelist=[1],default=0)
        df['CC_TRANSACTOR'] = np.select(condlist=[(df['bank_card_payment_category']=='TRANSACTOR')],choicelist=[1],default=0)
        df['CC_TRANPLUS'] = np.select(condlist=[(df['bank_card_payment_category']=='TRANPLUS')],choicelist=[1],default=0)
    except:
        print("errors")

    df['credithungry'] = np.select(condlist=[((df['UNSECURED_HIGH_CREDIT_SUM']<=75000)&(df['SECURED_HIGH_CREDIT_SUM']<=0)&\
                                            (df['no_of_deduped_inquiries_l6m']>=4))],choicelist=[1],default=0)

    df['no_of_open_cc_trades_rnkXno_of_business_general_trades_rnkXno_of_90p_accs_ever_rnkX3091']=np.select(condlist=[((df['no_of_open_cc_trades']>=2.0)&(df['no_of_open_cc_trades']<=21.0)&(df['no_of_business_general_trades']>=0.0)&(df['no_of_business_general_trades']<=0.0)&(df['no_of_90p_accs_ever']>=1.0)&(df['no_of_90p_accs_ever']<=11.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_cc_util_revolving_l1m_6020']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>39.22)&(df['max_aggregate_bankcard_utilisation_l12m']<=80.38)&(df['cc_util_revolving_l1m']>9.21)&(df['cc_util_revolving_l1m']<=538.69))],choicelist=[1],default=0)
    df['no_of_trades_pl_2w_gl_cd_worst_rating_l24m__unsecured_high_credit_sum_42']=np.select(condlist=[((df['no_of_trades_pl_2w_gl_cd']>10)&(df['no_of_trades_pl_2w_gl_cd']<=284)&(df['worst_rating_l24m']>-0.01)&(df['worst_rating_l24m']<=1)&(df['UNSECURED_HIGH_CREDIT_SUM']>520000)&(df['UNSECURED_HIGH_CREDIT_SUM']<=57875849))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXworst_rating_l3m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkX150']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=0.0)&(df['max_aggregate_bankcard_utilisation_l12m']<=34.5)&(df['worst_rating_l3m']>=1.0)&(df['worst_rating_l3m']<=1.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']>=10329.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']<=33080853.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_unsecured_high_credit_sum_3292']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>-6.01)&(df['max_aggregate_bankcard_utilisation_l12m']<=-1)&(df['UNSECURED_HIGH_CREDIT_SUM']>574040.33)&(df['UNSECURED_HIGH_CREDIT_SUM']<=1418204.33))],choicelist=[1],default=0)
    df['no_of_trades_total_cl_of_cc_trades_l12m_9968']=np.select(condlist=[((df['no_of_trades']>-0.01)&(df['no_of_trades']<=4)&(df['total_cl_of_cc_trades_l12m']>-5.01)&(df['total_cl_of_cc_trades_l12m']<=-1))],choicelist=[1],default=0)
    df['worst_rating_l3m_no_of_cc_trades_60P_l12m_7369']=np.select(condlist=[((df['worst_rating_l3m']>-0.01)&(df['worst_rating_l3m']<=1)&(df['no_of_cc_trades_60P_l12m']>-1.01)&(df['no_of_cc_trades_60P_l12m']<=0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkXsecured_balances_sum_rnkX447']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=38.85)&(df['max_aggregate_bankcard_utilisation_l12m']<=1027.37)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']>=-1.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']<=-1.0)&(df['SECURED_BALANCES_SUM']>=71607.0)&(df['SECURED_BALANCES_SUM']<=495403790.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_auto_trades_l12m_rnkXno_of_deduped_inquiries_l3m_rnkX6945']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=0.0)&(df['max_aggregate_bankcard_utilisation_l12m']<=38.84)&(df['total_bal_open_auto_trades_l12m']>=-3.0)&(df['total_bal_open_auto_trades_l12m']<=-1.0)&(df['no_of_deduped_inquiries_l3m']>=2.0)&(df['no_of_deduped_inquiries_l3m']<=2.0))],choicelist=[1],default=0)
    df['worst_rating_l24m_no_pl_sanctionedamt_lt30K_l36m_7642']=np.select(condlist=[((df['worst_rating_l24m']>1)&(df['worst_rating_l24m']<=1.5)&(df['no_pl_sanctionedamt_lt30K_l36m']>0)&(df['no_pl_sanctionedamt_lt30K_l36m']<=1))],choicelist=[1],default=0)
    df['worst_rating_l24m_no_of_cc_lt30p_l12m_541']=np.select(condlist=[((df['worst_rating_l24m']>1.5)&(df['worst_rating_l24m']<=9)&(df['no_of_cc_lt30p_l12m']>1)&(df['no_of_cc_lt30p_l12m']<=2))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_rnkXno_of_deduped_inquiries_l12m_rnkXno_of_30P_cc_l12m_rnkX6968']=np.select(condlist=[((df['no_of_60p_accs_ever']>=0.0)&(df['no_of_60p_accs_ever']<=0.0)&(df['no_of_deduped_inquiries_l12m']>=8.0)&(df['no_of_deduped_inquiries_l12m']<=73.0)&(df['no_of_30P_cc_l12m']>=1.0)&(df['no_of_30P_cc_l12m']<=3.0))],choicelist=[1],default=0)
    df['worst_rating_l3m_no_of_90p_accs_ever_6844']=np.select(condlist=[((df['worst_rating_l3m']>1)&(df['worst_rating_l3m']<=9)&(df['no_of_90p_accs_ever']>0)&(df['no_of_90p_accs_ever']<=16))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_no_of_30P_cc_l12m_8953']=np.select(condlist=[((df['no_of_60p_accs_ever']>-6.01)&(df['no_of_60p_accs_ever']<=0)&(df['no_of_30P_cc_l12m']>-1)&(df['no_of_30P_cc_l12m']<=0))],choicelist=[1],default=0)
    df['current_utilization_of_top_wallet_bankcard8'] = np.select(condlist=[((df['current_utilization_of_top_wallet_bankcard']>88)&(df['current_utilization_of_top_wallet_bankcard']<=600))],choicelist=[1],default=0)
    df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'] = np.select(condlist=[df['total_bal_open_trades_l12m']<=0],choicelist=[df['no_of_deduped_inquiries']],default=df['no_of_deduped_inquiries']/df['total_bal_open_trades_l12m'])
    df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'] = np.select(condlist=[df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']>0.0156,df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']<0.00000001],\
                                                        choicelist=[0.0156,0.00000001],default=df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'])

    df['cc_util_revolving_l1m_unsecured_high_credit_sum_6330']=np.select(condlist=[((df['cc_util_revolving_l1m']>-6.01)&(df['cc_util_revolving_l1m']<=-1)&(df['UNSECURED_HIGH_CREDIT_SUM']>-0.01)&(df['UNSECURED_HIGH_CREDIT_SUM']<=46297.17))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_rnkXno_of_open_cc_trades_rnkXno_tr_opened_l3m_rnkX1196']=np.select(condlist=[((df['no_of_60p_accs_ever']>=0.0)&(df['no_of_60p_accs_ever']<=0.0)&(df['no_of_open_cc_trades']>=0.0)&(df['no_of_open_cc_trades']<=1.0)&(df['no_tr_opened_l3m']>=2.0)&(df['no_tr_opened_l3m']<=22.0))],choicelist=[1],default=0)
    df['total_bal_open_trades_l12m_no_of_trades_pl_2w_gl_cd_4682']=np.select(condlist=[((df['total_bal_open_trades_l12m']>-5.01)&(df['total_bal_open_trades_l12m']<=28475.83)&(df['no_of_trades_pl_2w_gl_cd']>2)&(df['no_of_trades_pl_2w_gl_cd']<=4))],choicelist=[1],default=0)
    df['cc_util_revolving_l1m_rnkXno_of_deduped_inquiries_l3m_rnkXunsecured_high_credit_sum_rnkX8903']=np.select(condlist=[((df['cc_util_revolving_l1m']>=3.66)&(df['cc_util_revolving_l1m']<=148.68)&(df['no_of_deduped_inquiries_l3m']>=3.0)&(df['no_of_deduped_inquiries_l3m']<=31.0)&(df['UNSECURED_HIGH_CREDIT_SUM']>=123090.0)&(df['UNSECURED_HIGH_CREDIT_SUM']<=565217.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_no_of_deduped_inquiries_www']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>-6.01)&(df['max_aggregate_bankcard_utilisation_l12m']<=-2)&(df['no_of_deduped_inquiries']>6)&(df['no_of_deduped_inquiries']<=8))],choicelist=[1],default=0)
    df['months_since_oldest_trade_no_of_deduped_inquiries_l1m']=np.select(condlist=[((df['months_since_oldest_trade']>-5.01)&(df['months_since_oldest_trade']<=24)&(df['no_of_deduped_inquiries_l1m']>-0.01)&(df['no_of_deduped_inquiries_l1m']<=1))],choicelist=[1],default=0)
    df['no_of_90p_accs_ever_no_of_30P_cc_l24m_3957']=np.select(condlist=[((df['no_of_90p_accs_ever']>0)&(df['no_of_90p_accs_ever']<=16)&(df['no_of_30P_cc_l24m']>0)&(df['no_of_30P_cc_l24m']<=27))],choicelist=[1],default=0)
    df['total_bal_open_trades_l12m_worst_rating_l12m']=np.select(condlist=[((df['total_bal_open_trades_l12m']>72909.14)&(df['total_bal_open_trades_l12m']<=145061.71)&(df['worst_rating_l12m']>1)&(df['worst_rating_l12m']<=1.5))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXno_of_business_general_trades_rnkXworst_rating_l3m_rnkX571']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=42.59)&(df['max_aggregate_bankcard_utilisation_l12m']<=113.37)&(df['no_of_business_general_trades']>=1.0)&(df['no_of_business_general_trades']<=9.0)&(df['worst_rating_l3m']>=1.5)&(df['worst_rating_l3m']<=9.0))],choicelist=[1],default=0)
    df['no_of_trades_no_of_trades_pl_2w_gl_cd_8980']=np.select(condlist=[((df['no_of_trades']>24)&(df['no_of_trades']<=380)&(df['no_of_trades_pl_2w_gl_cd']>2)&(df['no_of_trades_pl_2w_gl_cd']<=4))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_cl_open_pl_trades_l12m_rnkXworst_rating_l3m_rnkX2111']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=39.1)&(df['max_aggregate_bankcard_utilisation_l12m']<=178.39)&(df['total_cl_open_pl_trades_l12m']>=3600.0)&(df['total_cl_open_pl_trades_l12m']<=200000.0)&(df['worst_rating_l3m']>=1.5)&(df['worst_rating_l3m']<=9.0))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_no_of_deduped_inquiries_l12m_4']=np.select(condlist=[((df['no_of_60p_accs_ever']>-6.01)&(df['no_of_60p_accs_ever']<=0)&(df['no_of_deduped_inquiries_l12m']>5)&(df['no_of_deduped_inquiries_l12m']<=7))],choicelist=[1],default=0)
    df['worst_rating_l24m_cc_util_revolving_l1m_1387']=np.select(condlist=[((df['worst_rating_l24m']>1.5)&(df['worst_rating_l24m']<=9)&(df['cc_util_revolving_l1m']>-6.01)&(df['cc_util_revolving_l1m']<=-1))],choicelist=[1],default=0)
    df['no_of_trades_total_bal_of_open_pl_l12m_6726']=np.select(condlist=[((df['no_of_trades']>15)&(df['no_of_trades']<=24)&(df['total_bal_of_open_pl_l12m']>247.67)&(df['total_bal_of_open_pl_l12m']<=51449))],choicelist=[1],default=0)
    df['no_of_open_cc_trades_total_bal_open_trades_l12m_5298']=np.select(condlist=[((df['no_of_open_cc_trades']>-1)&(df['no_of_open_cc_trades']<=1)&(df['total_bal_open_trades_l12m']>-5.01)&(df['total_bal_open_trades_l12m']<=28475.83))],choicelist=[1],default=0)
    return df

def getNapsScore(row):
    
    zz = (-1.7249)+(row['no_of_60p_accs_ever_no_of_30P_cc_l12m_8953']*-0.6572)+(row['worst_rating_l6m']*-0.3441)+(row['worst_rating_l24m_cc_util_revolving_l1m_1387']*0.8992)+\
         (row['no_of_open_cc_trades_rnkXno_of_business_general_trades_rnkXno_of_90p_accs_ever_rnkX3091']*1.0623)+(row['current_utilization_of_top_wallet_bankcard8']*0.6604)+(row['no_of_deduped_inquiries_l6m']*0.0361)+\
         (row['max_aggregate_bankcard_utilisation_l12m_cc_util_revolving_l1m_6020']*-0.8515)+(row['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']*31.4963)+(row['no_of_60p_accs_ever_no_of_deduped_inquiries_l12m_4']*-0.452)+\
         (row['no_of_trades_pl_2w_gl_cd_worst_rating_l24m__unsecured_high_credit_sum_42']*-0.7234)+(row['cc_util_revolving_l1m_unsecured_high_credit_sum_6330']*0.3369)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXworst_rating_l3m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkX150']*2.9448)+(row['no_of_60p_accs_ever_rnkXno_of_open_cc_trades_rnkXno_tr_opened_l3m_rnkX1196']*0.6413)+\
         (row['max_aggregate_bankcard_utilisation_l12m_unsecured_high_credit_sum_3292']*-0.7401)+(row['total_bal_open_trades_l12m_no_of_trades_pl_2w_gl_cd_4682']*-0.5306)+(row['no_of_trades_total_bal_of_open_pl_l12m_6726']*-1.0659)+\
         (row['no_of_trades_total_cl_of_cc_trades_l12m_9968']*0.3131)+(row['cc_util_revolving_l1m_rnkXno_of_deduped_inquiries_l3m_rnkXunsecured_high_credit_sum_rnkX8903']*1.3062)+\
         (row['worst_rating_l3m_no_of_cc_trades_60P_l12m_7369']*-0.6157)+(row['max_aggregate_bankcard_utilisation_l12m_no_of_deduped_inquiries_www']*0.8942)+(row['no_of_open_cc_trades_total_bal_open_trades_l12m_5298']*0.5325)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkXsecured_balances_sum_rnkX447']*-0.4146)+(row['months_since_oldest_trade_no_of_deduped_inquiries_l1m']*0.2631)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_auto_trades_l12m_rnkXno_of_deduped_inquiries_l3m_rnkX6945']*-1.5711)+(row['no_of_90p_accs_ever_no_of_30P_cc_l24m_3957']*-1.1005)+\
         (row['worst_rating_l24m_no_pl_sanctionedamt_lt30K_l36m_7642']*0.472)+(row['total_bal_open_trades_l12m_worst_rating_l12m']*0.6542)+(\
             row['worst_rating_l24m_no_of_cc_lt30p_l12m_541']*0.8891)+(row['max_aggregate_bankcard_utilisation_l12m_rnkXno_of_business_general_trades_rnkXworst_rating_l3m_rnkX571']*1.6077)+\
             (row['no_of_60p_accs_ever_rnkXno_of_deduped_inquiries_l12m_rnkXno_of_30P_cc_l12m_rnkX6968']*1.6329)+(row['no_of_trades_no_of_trades_pl_2w_gl_cd_8980']*1.9295)+\
             (row['worst_rating_l3m_no_of_90p_accs_ever_6844']*0.9481)+(row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_cl_open_pl_trades_l12m_rnkXworst_rating_l3m_rnkX2111']*1.0015)

    r = np.exp(zz)/(1+np.exp(zz))
    scr = int(max(350,min(575*(r**(-0.08)),900)))

    return scr

scrub_data['SECURED_ACCOUNTS_COUNT']=scrub_data['Secured Accounts Count']
scrub_data['UNSECURED_ACCOUNTS_COUNT']=scrub_data['Unsecured Accounts Count']
scrub_data['SECURED_HIGH_CREDIT_SUM']=scrub_data['Secured High Credit Sum']
scrub_data['UNSECURED_HIGH_CREDIT_SUM']=scrub_data['Unsecured High Credit Sum']
scrub_data['SECURED_AMOUNT_OVERDUE_SUM']=scrub_data['Secured Amount Overdue Sum']
scrub_data['UNSECURED_AMOUNT_OVERDUE_SUM']=scrub_data['Unsecured Amount Overdue Sum']
scrub_data['SECURED_BALANCES_SUM']=scrub_data['Secured Balances Sum']
scrub_data['UNSECURED_BALANCES_SUM']=scrub_data['Unsecured Balances Sum']
              
scrub_data = preprocessor(scrub_data)
scrub_data['naps_new']= scrub_data.apply(getNapsScore,axis=1)
scrub_data['naps_new_bkt'] = scrub_data.apply(naps_score_bkt,var='naps_new',axis=1)

In [ ]:
cols = ['Member Reference', 'Mobile_No',
       'max_aggregate_bankcard_utilisation_l12m', 'cc_util_revolving_l1m',
       'bank_card_payment_category', 'no_of_60p_accs_ever',
       'no_of_deduped_inquiries', 'total_cl_open_mortgage_trades_l12m',
       'total_balance_open_mortgage_trades_l12m', 'total_bal_of_open_pl_l12m',
       'months_since_oldest_trade', 'no_of_mortgage_trades',
       'no_of_open_cc_trades', 'no_of_business_general_trades',
       'no_of_30p_accs_ever', 'no_of_trades', 'total_bal_open_trades_l12m',
       'total_bal_open_auto_trades_l12m', 'months_since_recent_chargedoff180P',
       'total_cl_open_auto_trades_l12m', 'total_cl_open_pl_trades_l12m',
       'no_of_chargedoff180P', 'total_cl_of_cc_trades_l12m',
       'no_of_90p_accs_ever', 'total_bal_of_chargedoff_trades',
       'worst_rating_l12m', 'current_utilization_of_top_wallet_bankcard',
       'no_of_trades_pl_2w_gl_cd',
       'total_bal_open_trades_l12m_except_wo_cc_od',
       'no_of_deduped_inquiries_l12m', 'no_of_deduped_inquiries_l6m',
       'no_of_deduped_inquiries_l3m', 'no_of_deduped_inquiries_l1m',
       'worst_rating_l24m', 'worst_rating_l6m', 'worst_rating_l3m',
       'worst_rating_l1m', 'no_of_60p_accs_l24m', 'no_of_60p_accs_l12m',
       'no_of_trades_bounced_l36m', 'no_of_trades_bounced_l12m',
       'no_of_cc_trades_60P_l24m', 'no_of_30P_cc_l24m',
       'no_of_cc_trades_60P_l12m', 'no_of_30P_cc_l12m', 'no_of_cc_lt30p_l12m',
       'no_cc_sanctionedamt_lt30K_l36m', 'no_pl_sanctionedamt_lt30K_l36m',
       'no_tr_opened_l6m', 'worst_dpd_l36m',
       'total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades',
       'no_tr_opened_l12m', 'no_tr_opened_l3m', 'per_of_30p_accs_ever',
       'total_payment_amount_bankcard_accs_l3m',
       'max_revolving_monthly_spend_l12m', 'total_monthly_obligations_l3m',
       'total_balance_bankcard_accs_l3m', 'no_of_trades_unsec',
       'total_payment_amount_l3m', 'worst_dpd_l2m', 'Secured Accounts Count',
       'Unsecured Accounts Count', 'Secured High Credit Sum',
       'Unsecured High Credit Sum', 'Secured Amount Overdue Sum',
       'Unsecured Amount Overdue Sum', 'Secured Balances Sum',
       'Unsecured Balances Sum', 'Own Accounts count', 'Other Accounts count',
       'CIBIL_score', 
       'no_of_open_cc_trades_bkt', 'years_on_bureau_bkt',
       'no_of_deduped_inquiries_bkt', 'Secured Loan_bkt',
       'cust_category', 'cust_hirisk', 'CC_INACTIVE', 'CC_NOBC', 'CC_REVOLVER',
       'CC_RVLRPLUS', 'CC_TRANSACTOR', 'CC_TRANPLUS', 'credithungry', 'naps_new', 'naps_new_bkt']

In [ ]:
scrub_data = scrub_data[cols]

In [ ]:
scrub_data.head()

In [ ]:
print('Before merging: ', master_data.shape)
master_data = pd.merge(master_data.rename(columns = {'Phone Numbers':'Mobile_No'}), scrub_data, on = 'Mobile_No', how = 'inner')
print('After merging: ', master_data.shape)

In [ ]:
master_data.head()

#### Derived variables

In [ ]:
master_data.columns = master_data.columns.str.replace('[^a-zA-Z0-9]', '_')
master_data.columns = master_data.columns.str.replace(' ', '_') 
master_data.columns = master_data.columns.str.lower()

In [ ]:
master_data['state'] = master_data['state'].replace(0,None)

In [ ]:
master_data['category'] = np.where(master_data['category'].isin(["CAT-A", "CAT-B"]), "CAT-AB", master_data['category'])

In [ ]:
master_data['category'].value_counts()

In [ ]:
master_data.head()

In [ ]:
state_region_mapping = {
    'TAMIL NADU': 'South',
    'ANDHRA PRADESH': 'South',
    'KERALA': 'South',
    'KARNATAKA': 'South',
    'MAHARASHTRA': 'West',
    'MADHYA PRADESH': 'Central',
    'DELHI': 'North',
    'UTTAR PRADESH': 'North',
    'PUNJAB': 'North',
    'WEST BENGAL': 'East',
    'GUJARAT': 'West',
    'HARYANA': 'North',
    'RAJASTHAN': 'North',
    'ODISHA': 'East',
    'UTTARAKHAND': 'North',
    'JHARKHAND': 'East',
    'CHANDIGARH': 'North',
    'CHATTISGARH': 'Central',
    'PONDICHERRY': 'South',
    'GOA': 'West',
    'BIHAR': 'East',
    'DADRA & NAGAR HAVELI': 'West',
    'HIMACHAL PRADESH': 'North',
    'DAMAN & DIU': 'West'
}


In [ ]:
master_data['region'] = master_data['state'].map(state_region_mapping)

In [ ]:
master_data['pos_by_la'] = master_data['month_pos'] / master_data['disbursed_amount']
master_data['offered_la_by_disb_la'] = master_data['max_amount'] / master_data['disbursed_amount']
master_data['emi_by_income'] = master_data['monthly_emi'] / master_data['monthly_income_(net_take_home)']
master_data['naps_diff'] = master_data.naps_new - master_data.naps_disb_score
master_data['naps_diff_per'] = (master_data.naps_new - master_data.naps_disb_score) / master_data.naps_disb_score
master_data['mob_by_tenure'] = master_data['months_since_disb'] / master_data['tenor_in_months']

In [ ]:
master_data = master_data.apply(pd.to_numeric, errors='ignore')
master_data['monthly_income_(net_take_home)'] = master_data['monthly_income_(net_take_home)'].astype(int)

In [ ]:
master_data['interest_rate_bkt'] = np.where(master_data['interest_rate'] <= 19, 'A) <=19',
                                   np.where(master_data['interest_rate'] <= 21, 'B) 19-21',
                                   np.where(master_data['interest_rate'] <= 24, 'C) 21-24',
                                   np.where(master_data['interest_rate'] <= 26, 'D) 24-26',
                                   np.where(master_data['interest_rate'] <= 28, 'E) 24-28',
                                   np.where(master_data['interest_rate'] > 28, 'F) 28+', None))))))

In [ ]:
master_data.drop(columns = ['createdate', 'member_reference', 'lender_loan_id', 'niro_user_id', 'disbursement_date', 'bureau_city', 'member_reference', 'city'], inplace=True)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming master_data is your DataFrame
# Replace 'YourCategoricalColumn' and 'YourNumericalColumn' with the actual column names

# Display value counts for categorical columns
categorical_columns = master_data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f"Value counts for {col}:")
    print(master_data[col].value_counts())
    print("\n")

# Display separate histograms for numerical columns
numerical_columns = master_data.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_columns:
    plt.figure(figsize=(8, 6))
    master_data[col].hist(bins=20)
    plt.title(f'Histogram for {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
master_data.to_pickle('master_data.pkl')